In [76]:
import json
import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
json_file = 'hci-charts/hci-charts-manual-filtered.json'
reference_file = 'concadia-charts/concadia-charts-manual-filtered.json'

In [3]:
df = pd.read_json(json_file)

In [4]:
df

,images
0,{'title': 'Blind people and mobile touch-based...
1,{'title': 'Blind people and mobile touch-based...
2,{'title': 'Blind people and mobile touch-based...
3,{'title': 'Blind people and mobile touch-based...
4,{'title': 'Blind people and mobile touch-based...
...,...
370,{'title': 'MagTouch: Robust Finger Identificat...
371,{'title': 'ToonNote: Improving Communication i...
372,{'title': 'Effect of target size on non-visual...
373,{'title': 'Effect of target size on non-visual...


In [5]:
df_ref = pd.read_json(reference_file)

In [6]:
df_ref

,images
0,"{'article_id': 'Autism', 'filename': '4.jpg', ..."
1,"{'article_id': 'Bulgaria', 'filename': '597.jp..."
2,"{'article_id': 'Brain', 'filename': '644.jpg',..."
3,"{'article_id': 'British Rail', 'filename': '79..."
4,"{'article_id': 'Dubnium', 'filename': '1262.jp..."
...,...
166,"{'article_id': 'Unemployment in South Korea', ..."
167,{'article_id': 'Matchbox Educable Noughts and ...
168,{'article_id': 'Colonial police action against...
169,"{'article_id': 'Minecraft (franchise)', 'filen..."


In [7]:
for key, value in df_ref['images'].iloc[0].items():
    print(key, ":",value)

article_id : Autism
filename : 4.jpg
orig_filename : https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/US-autism-6-17-1996-2007.png/220px-US-autism-6-17-1996-2007.png
description : {'raw': 'Bar chart versus time. The graph rises steadily from 1996 to 2007, from about 0.7 to about 5.3. The trend curves slightly upward.', 'tokens': ['Bar', 'chart', 'versus', 'time', '.', 'The', 'graph', 'rises', 'steadily', 'from', '1996', 'to', '2007', ',', 'from', 'about', '0.7', 'to', 'about', '5.3', '.', 'The', 'trend', 'curves', 'slightly', 'upward', '.']}
caption : {'raw': 'Reports of autism cases per 1,000 children grew dramatically in the US from 1996 to 2007. It is unknown how much, if any, growth came from changes in rates of autism.', 'tokens': ['Reports', 'of', 'autism', 'cases', 'per', '1,000', 'children', 'grew', 'dramatically', 'in', 'the', 'US', 'from', '1996', 'to', '2007', '.', 'It', 'is', 'unknown', 'how', 'much', ',', 'if', 'any', ',', 'growth', 'came', 'from', 'changes', 'in'

In [8]:
for key, value in df['images'].iloc[0].items():
    print(key, ":",value)

title : Blind people and mobile touch-based text-entry: acknowledging the need for different flavors
pdf_hash : 3b7898e31176cfe6b7e2b6309e2f18c63103baa9
year : 2011
venue : ASSETS
alt_text : This figure presents a collum chart with the average WPM for each method. QWERTY ist the fastest followed by MultiTap, NavTouch and BrailleTouch.
levels : [[1], [2]]
corpus_id : 3187042
sentences : ['This figure presents a collum chart with the average WPM for each method.', 'QWERTY ist the fastest followed by MultiTap, NavTouch and BrailleTouch.']
caption : Figure 4. WPM (average) across the different methods. Error bars denote 95% CI.
local_uri : ['3b7898e31176cfe6b7e2b6309e2f18c63103baa9_Image_005.jpg']
annotated : True
is_plot : True
uniq_levels : [1, 2]
has_1_2_3 : False
has_1_2 : True
compound : False


In [59]:
mapping = {'title': 'article_id', 
           'alt_text':['description'], 
           'caption': ['caption'], 
           'local_uri':'filename',
           'pdf_hash': 'orig_filename'}

In [69]:
new_json = []
for row in df['images'].items():
    entry = row[1]
    reformat_entry = {}
    for key, value in mapping.items():
        if type(value) is not list:
            reformat_entry[value] = entry[key] #TODO: split images later
        else:
            outer_value = entry[key]
            reformat_entry[value[0]] = {'raw': outer_value, 'tokens': word_tokenize(outer_value)}
            if value[0]=='caption':
                context = entry['title'] + " "+ outer_value 
                reformat_entry['context'] = {'raw': context, 'tokens': word_tokenize(context)}
    new_json.append(reformat_entry)

In [70]:
new_json[0]

{'article_id': 'Blind people and mobile touch-based text-entry: acknowledging the need for different flavors',
 'description': {'raw': 'This figure presents a collum chart with the average WPM for each method. QWERTY ist the fastest followed by MultiTap, NavTouch and BrailleTouch.',
  'tokens': ['This',
   'figure',
   'presents',
   'a',
   'collum',
   'chart',
   'with',
   'the',
   'average',
   'WPM',
   'for',
   'each',
   'method',
   '.',
   'QWERTY',
   'ist',
   'the',
   'fastest',
   'followed',
   'by',
   'MultiTap',
   ',',
   'NavTouch',
   'and',
   'BrailleTouch',
   '.']},
 'caption': {'raw': 'Figure 4. WPM (average) across the different methods. Error bars denote 95% CI.',
  'tokens': ['Figure',
   '4',
   '.',
   'WPM',
   '(',
   'average',
   ')',
   'across',
   'the',
   'different',
   'methods',
   '.',
   'Error',
   'bars',
   'denote',
   '95',
   '%',
   'CI',
   '.']},
 'context': {'raw': 'Blind people and mobile touch-based text-entry: acknowledging t

In [71]:
split_json = []
for entry in new_json:
    for file in entry['filename']:
        modified_entry = entry.copy()
        modified_entry['filename'] = file
        split_json.append(modified_entry)

In [73]:
len(split_json)

398

In [74]:
split_json[0]

{'article_id': 'Blind people and mobile touch-based text-entry: acknowledging the need for different flavors',
 'description': {'raw': 'This figure presents a collum chart with the average WPM for each method. QWERTY ist the fastest followed by MultiTap, NavTouch and BrailleTouch.',
  'tokens': ['This',
   'figure',
   'presents',
   'a',
   'collum',
   'chart',
   'with',
   'the',
   'average',
   'WPM',
   'for',
   'each',
   'method',
   '.',
   'QWERTY',
   'ist',
   'the',
   'fastest',
   'followed',
   'by',
   'MultiTap',
   ',',
   'NavTouch',
   'and',
   'BrailleTouch',
   '.']},
 'caption': {'raw': 'Figure 4. WPM (average) across the different methods. Error bars denote 95% CI.',
  'tokens': ['Figure',
   '4',
   '.',
   'WPM',
   '(',
   'average',
   ')',
   'across',
   'the',
   'different',
   'methods',
   '.',
   'Error',
   'bars',
   'denote',
   '95',
   '%',
   'CI',
   '.']},
 'context': {'raw': 'Blind people and mobile touch-based text-entry: acknowledging t

In [77]:
with open("hci-charts/context-hci.json", "w") as outfile:
    json_object = {'images': split_json}
    outfile.write(json.dumps(json_object))